# Introduction to Human Activity Recognition

Here we will explore some common ways of preprocessing human activity recognition data.

Using the example data we will learn:
* how to merge multiple files into one large DataFrame
* how to split data into sliding windows
* how to quickly extract features from a window
* how to set the number of classes considered for classification
* how to build a simple Random Forest Classifier and train it on HAR data
* how to build a simple CNN and train it on HAR data 

Bear in mind that the sample data offered is not cleaned or high quality. You should not use it in your own experiments but it is useful for this tutorial.

You will need the following packages: 
* tsfresh
* scikit-learn
* tensorflow

## Current Model Note for cw3 

#### Basic imports

In [222]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf

import os

import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K

## Loading multiple files into one large DataFrame

At this stage you should only be working with clean data, saved in the format required for Coursework 1. An example of such data can be found in the Data/Clean/ folder.

In [368]:
base_df = pd.DataFrame()

clean_data_folder = "../pdiot-data/2021"

for sUNN_folder in os.listdir(clean_data_folder):
    print(sUNN_folder)
    try:
        for filename in os.listdir(clean_data_folder+"/"+sUNN_folder):
            
            full_path = f"{clean_data_folder}/{sUNN_folder}/{filename}"
#             print(full_path)

            # load data into a DataFrame
            new_df = pd.read_csv(full_path)

            # merge into the base DataFrame
            base_df = pd.concat([base_df, new_df])
    except:
        print(sUNN_folder, " is not a folder")

s1870467
s1870697
s2171931
s2254050
s1727780
s2125423
.DS_Store
.DS_Store  is not a folder
s1757177
s1702583
s1824891
s1826390
s1865890
s1823274
s1865457
s1800883
s1718069
s1706154
s1864705
s1817455
s2211162
s1761322
s1970333
s2192970
s2181154
s1711661
s2250677
s1893474
s2119637
s1721256
s1704145
s1894401
s1541031
s1843072
s1893835
README.md
README.md  is not a folder
s2173036
s2212045
s1813106
s1891214
s1724279
s1822958
s1841064
s2171825
s1714206
s1801931
s1842093
s1724067
s1732873
s1817972
s1850642
s2211228


Reset the index of the base DataFrame.

In [390]:
#Use only respeck data. Also, remove columns that are added by mistake 
required_columns = ["timestamp","accel_x","accel_y","accel_z","gyro_x","gyro_y","gyro_z","sensor_type","activity_type", "activity_code", "subject_id","recording_id"]
base_df.reset_index(drop=True, inplace=True)
base_df = base_df[required_columns]
base_df = base_df[base_df.sensor_type=="Respeck"] 
base_df.reset_index(drop=True, inplace=True)


base_df.dropna(inplace=True)
base_df

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,sensor_type,activity_type,activity_code,subject_id,recording_id
0,1.633516e+12,0.261475,-1.116516,-0.502991,-0.812500,12.312500,19.500000,Respeck,Climbing stairs,12,s1870467,Respeck_s1870467_Climbing stairs_06-10-2021_11...
1,1.633516e+12,-0.177734,-0.636292,-0.477600,-5.265625,2.953125,-9.281250,Respeck,Climbing stairs,12,s1870467,Respeck_s1870467_Climbing stairs_06-10-2021_11...
2,1.633516e+12,0.351562,-0.879456,-0.335754,5.671875,24.656250,-10.562500,Respeck,Climbing stairs,12,s1870467,Respeck_s1870467_Climbing stairs_06-10-2021_11...
3,1.633516e+12,0.298584,-1.253479,-0.299622,-3.609375,2.687500,5.890625,Respeck,Climbing stairs,12,s1870467,Respeck_s1870467_Climbing stairs_06-10-2021_11...
4,1.633516e+12,0.056152,-0.777405,-0.210754,-14.328125,5.421875,8.140625,Respeck,Climbing stairs,12,s1870467,Respeck_s1870467_Climbing stairs_06-10-2021_11...
...,...,...,...,...,...,...,...,...,...,...,...,...
673313,1.632930e+12,-0.376465,-0.746399,0.693542,-0.171875,-0.953125,1.281250,Respeck,Sitting bent backward,5,s2211228,Respeck_s2211228_Sitting bent backward_29-09-2...
673314,1.632930e+12,-0.345703,-0.749329,0.677429,-0.671875,-0.187500,-1.015625,Respeck,Sitting bent backward,5,s2211228,Respeck_s2211228_Sitting bent backward_29-09-2...
673315,1.632930e+12,-0.368408,-0.745422,0.690857,-0.218750,-0.640625,0.375000,Respeck,Sitting bent backward,5,s2211228,Respeck_s2211228_Sitting bent backward_29-09-2...
673316,1.632930e+12,-0.363281,-0.747375,0.689148,0.078125,0.796875,-0.046875,Respeck,Sitting bent backward,5,s2211228,Respeck_s2211228_Sitting bent backward_29-09-2...


Now you can get a list of all recording ids, activities, sensor types and anything else you might need.

In [380]:
print(f"The data was collected using the sensors: {base_df.sensor_type.unique()}")
print(f"The data was collected for the activities: {base_df.activity_type.unique()}")
print(f"The number of unique recordings is: {len(base_df.recording_id.unique())}")
print(f"The subject IDs in the recordings are: {len(base_df.subject_id.unique())}")
print(len(base_df.activity_type.unique()))

The data was collected using the sensors: ['Respeck']
The data was collected for the activities: ['Climbing stairs' 'Standing' 'Falling on the back' 'Movement'
 'Lying down on stomach' 'Sitting bent backward' 'Lying down left'
 'Lying down on back' 'Descending stairs' 'Sitting bent forward'
 'Walking at normal speed' 'Running' 'Falling on the left'
 'Lying down right' 'Falling on the right' 'Desk work' 'Falling on knees'
 'Sitting']
The number of unique recordings is: 3488
The subject IDs in the recordings are: 48
18


You can of course change the clean data folder to where you keep all the PDIoT data and you should be seeing a lot more subject IDs, unique recordings and activity types.

## Splitting data into sliding windows

The sliding window approach is one of the most efficient ways to process Human Activity Recognition data. We saw in the last notebook that sensor data comes in the form of Time Series. One single datapoint is not enough to represent an activity, we need a larger snapshot of the signal for that. The image below shows how a sliding window achieves that. 

![sliding windows](../Images/sliding_windows_complete.png "Sliding Windows")

The windows can have some amount of overlap, as shown in the picture, or they can have no overlap at all in which case they would be side-by-side.

Each window can now be treated as an input datapoint to whichever model you choose to train. 

### Watch out for separate recordings

You need to make sure that when you split your data into sliding windows you don't accidentally include two separate recordings in the same window. This would cause the signal from the first recording to suddenly "jump" to an unrelated value from the second recording. 

For this, you will have to first split up your dataset by recording (this is where the recording ID comes in), then you have to split each recording into sliding windows. At the end you can aggregate all of your resulting sliding windows in a large dataset.

In [235]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [236]:
np.array(first_four_windows[0][columns_of_interest]).shape

(50, 6)

## From sliding windows to datapoints

Each sliding window needs to be further processed in order to represent an appropriate input datapoint. The preprocessing methods might differ depending on which type of model you choose to use. 

You can also do further processing on the signal types and axes, for example smooth the signal, apply axis fusion, eliminate noise etc. 

Here we will discuss simple examples without any preprocessing. 

### Feature extraction with tsfresh

One quick and simple method to consider for HAR is a [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). It can take a multi-dimensional datapoint as input and can output a classification. 

We can transform each window of data into one single, multidimensional datapoint by applying simple computations using the [tsfresh](https://tsfresh.readthedocs.io/en/latest/text/feature_extraction_settings.html) package.

Now the index represents the window ID, and each row in the DataFrame represents a multi-dimensional datapoint which we can use as input to the RFC.

You can use the window ID to refer back to the initial dataframe and get the class (activity type) for each window.

We will now process both recordings so that we have two classes for our classifier. For your own work you should only aim to perform classification on data from one sensor. 

In [477]:
window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

# copy_df = df
# copy_df.reset_index(drop=True, inplace=True)
# copy_df = copy_df[required_columns]
# copy_df = copy_df[copy_df.sensor_type=="Respeck"] 
# copy_df.reset_index(drop=True, inplace=True)
# print(only_respeck_df[only_respeck_df.recording_id=="Respeck_s1541031_Falling on knees_06-10-2021_15-46-54"])

for rid, group in base_df.groupby("activity_code"):
    print(f"Processing rid = {rid}")
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]



    overlapping_windows = large_enough_windows[::step_size] 
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1


    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))

Processing rid = 0
Processing rid = 1
Processing rid = 2
Processing rid = 4
Processing rid = 5
Processing rid = 6
Processing rid = 7
Processing rid = 8
Processing rid = 9
Processing rid = 11
Processing rid = 12
Processing rid = 13
Processing rid = 31
Processing rid = 45
Processing rid = 46
Processing rid = 47
Processing rid = 48
Processing rid = 100


In [491]:
final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)

is_NaN = final_sliding_windows.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = final_sliding_windows[row_has_NaN]

print(rows_with_NaN)

Empty DataFrame
Columns: [timestamp, accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, sensor_type, activity_type, activity_code, subject_id, recording_id, window_id]
Index: []


We need to assign a number to each categorical class for the Random Forest Classifer. It is mainly up to you how you categorise your classes. In this example, we will use the labels:
- 0 for Desk work
- 1 for Walking at normal speed

Bear in mind that your classification task will be multi-class, not binary. 

In [479]:
class_labels = {}
label_to_activity = {}
activities = sorted(final_sliding_windows.activity_type.unique())

for idx, activity in enumerate(activities):
    class_labels[activity] = idx
    label_to_activity[idx] = activity

print(class_labels)

{'Climbing stairs': 0, 'Descending stairs': 1, 'Desk work': 2, 'Falling on knees': 3, 'Falling on the back': 4, 'Falling on the left': 5, 'Falling on the right': 6, 'Lying down left': 7, 'Lying down on back': 8, 'Lying down on stomach': 9, 'Lying down right': 10, 'Movement': 11, 'Running': 12, 'Sitting': 13, 'Sitting bent backward': 14, 'Sitting bent forward': 15, 'Standing': 16, 'Walking at normal speed': 17}


In [480]:
window_id_class_labels = final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min).replace(class_labels)
window_id_class_labels

,activity_type
window_id,
0,13
1,13
2,13
3,13
4,13
...,...
67302,16
67303,16
67304,16


## Leave-one-subject-out cross-validation

One of the other reason our previous classifier functions so well is that each person performs activities in a very consistent manner. If a subject's data is both in the training set and the test set, it provides the model with an unfair advantage. Moreover, the results that your model will output will be falsely high. When you test your model on a completely new, unseen subject, your accuracy will drop considerably. 

This is why when training a HAR model you will want to do a special kind of cross-validation: Leave-One-Subject-Out (LOSOXV), where we leave one (or more) subject(s) in the testing set at each iteration.

![losoxv](../Images/LOOCV.png "losoxv")

This ensures that the results we get from our classifier are consistent to what we would get in real life, if we were to test the model on a new user.

## Training a simple CNN

There are some different preprocessing techniques you can apply when the resulting "datapoint" is an input to a convolutional neural network. 

You can use two types of convolutional layers:
* 1D Conv Layers - which will work on 1D data, for example a single axis from one single sensor (accel, gyro or mag)
* 2D Conv Layers - suitable if the input data is in the form of an image, for example

We will be demonstrating how to build a simple 1D CNN using 6 channels: the 3 axes of the accelerometer and the 3 axes of the gyroscope.

### Setting up the model

In [482]:
filters = 64
kernel_size = 3
n_features = 6
activation='relu'
n_classes = len(class_labels)
print(n_classes)

18


In [483]:
model = Sequential()

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear', 
                 input_shape=(window_size, n_features)))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 48, 64)            1216      
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 64)            256       
_________________________________________________________________
activation_21 (Activation)   (None, 48, 64)            0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 46, 64)            12352     
_________________________________________________________________
batch_normalization_22 (Batc (None, 46, 64)            256       
_________________________________________________________________
activation_22 (Activation)   (None, 46, 64)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 44, 64)           

### Re-generating the data in the appropriate format

We need to do a bit more work with our data to bring it into a format fit for training a CNN. 

A CNN will take multi-dimensional arrays as input. We have already specified that the input shape is (window_size, n_features), i.e. (50, 6). 

Remember that we generated sliding windows before. Now we just need to take the raw values from each window and create a training set. 

In [325]:
columns_of_interest

['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [492]:
X = []
y = []

for window_id, group in final_sliding_windows.groupby('window_id'):
#     print(f"window_id = {window_id}")
    
    shape = group[columns_of_interest].values.shape
#     print(f"shape = {shape}")
    
    X.append(group[columns_of_interest].values)
    y.append(class_labels[group["activity_type"].values[0]])

In [493]:
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

X shape = (26907, 50, 6)
y shape = (26907,)


We can do a similar test/train split for demonstration purposes. Remember that you will have to split your data by subjects, not radomly.

In [494]:

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, train_size=0.8)

y_train = np.asarray(pd.get_dummies(y_train), dtype=np.float32)
y_test = np.asarray(pd.get_dummies(y_test), dtype=np.float32)

In [495]:
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")

print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")



X_train shape = (21525, 50, 6)
y_train shape = (21525, 18)
X_test shape = (5382, 50, 6)
y_test shape = (5382, 18)


Now we are ready to start the training process

In [496]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(
    optimizer=optimizers.SGD(lr=0.01),
    loss='categorical_crossentropy',
    metrics = ['accuracy'])

In [497]:
model.fit(X_train, y_train,
        batch_size=128, epochs=20, callbacks=[callback])

Train on 21525 samples
Epoch 1/20
21525/21525 [==============================] - 35s 2ms/sample - loss: 2.2701 - accuracy: 0.2645
Epoch 2/20
21525/21525 [==============================] - 35s 2ms/sample - loss: 1.7688 - accuracy: 0.4519
Epoch 3/20
21525/21525 [==============================] - 34s 2ms/sample - loss: 1.4265 - accuracy: 0.5511
Epoch 4/20
21525/21525 [==============================] - 32s 2ms/sample - loss: 1.2433 - accuracy: 0.6059
Epoch 5/20
21525/21525 [==============================] - 31s 1ms/sample - loss: 1.1339 - accuracy: 0.6300
Epoch 6/20
21525/21525 [==============================] - 35s 2ms/sample - loss: 1.0529 - accuracy: 0.6428
Epoch 7/20
21525/21525 [==============================] - 28s 1ms/sample - loss: 1.0024 - accuracy: 0.6562
Epoch 8/20
21525/21525 [==============================] - 29s 1ms/sample - loss: 0.9540 - accuracy: 0.6656
Epoch 9/20
21525/21525 [==============================] - 29s 1ms/sample - loss: 0.9165 - accuracy: 0.6736
Epoch 10/20
21

We can now view the accuracy of our model on the test dataset.

In [498]:
# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print(y_pred_labels)

[16  7 15 ...  4  4  5]


In [499]:
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       0.67      0.30      0.41       272
           1       0.60      0.44      0.51       296
           2       0.73      0.28      0.41       277
           3       0.83      0.72      0.77       283
           4       0.86      0.80      0.82       299
           5       0.79      0.85      0.82       299
           6       0.81      0.78      0.80       286
           7       0.75      0.86      0.80       293
           8       0.81      0.85      0.83       317
           9       0.82      0.76      0.79       307
          10       0.84      0.87      0.86       318
          11       0.75      0.45      0.56       332
          12       0.93      0.91      0.92       266
          13       0.35      0.37      0.36       270
     

## Conclusion

You are now ready to start developing your own models for HAR. There are numerous tutorials online which you can follow to build models like LSTMs, CNNs, RFCs and many others. 

You have a wide choice of ways to solve this classification model. Here are a few things to think about:

* What type of preprocessing do you want to apply to your data? Examples include:
    * smoothing the sensor axes
    * performing axis fusion
    * extracting scalograms from the signal
    * manually extracting features from the signal
    * choosing to leave out certain axes

* What type of model do you want to train?
    * simple ML model
    * deep learning model
    
* Do you want a hierarchical model or a flat model?
    * hierarchical models means you don't have to train the same type of model for each activity
    * a flat model might be faster to train and apply in real time

## Save Model

In [500]:
models_directory = './models/'
current_model_path = models_directory + 'CNN_model_HARv_v1/'

tflite_model_filename = 'CNN_HAR_v1.tflite'

In [501]:
#Save original model first. We will use the SavedModel to convert it to TFLite as recommended by the Tensorflow documentation.
tf.saved_model.save(model, current_model_path)

INFO:tensorflow:Assets written to: ./models/CNN_model_HARv_v1/assets


INFO:tensorflow:Assets written to: ./models/CNN_model_HARv_v1/assets


## Convert SavedModel to TFLite

In [502]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(current_model_path) # path to the SavedModel directory
tflite_model = converter.convert()
tflite_model_filename = 'CNN_HAR_v1.tflite'
# Save the model.
with open(current_model_path+tflite_model_filename, 'wb') as f:
  f.write(tflite_model)

## Quick Test on Python - Tensorflow Lite




In [505]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=current_model_path+tflite_model_filename)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
print("Input Shape")
print(input_shape)
test = X_test.astype(np.float32)
#Test data to feed as parameter
test_part = test[2:3]
print("See Test Shape")
print(test_part.shape)

interpreter.set_tensor(input_details[0]['index'], test_part)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
output_results = np.squeeze(output_data)

print("Current output results")
print(output_results)
print(len(output_results))

print("Which is the most confident?")
max_index = np.argmax(output_results, axis=0)

print(max_index)



Input Shape
[ 1 50  6]
See Test Shape
(1, 50, 6)
Current output results
[6.2270577e-05 2.4624990e-06 1.5130171e-01 3.9093671e-04 5.0324547e-06
 7.4569823e-04 3.8591963e-05 1.3974484e-03 2.7534372e-06 2.3628731e-07
 1.2048521e-04 8.6326441e-03 9.0560906e-07 2.8892693e-01 3.9909825e-02
 4.7166407e-01 3.6739558e-02 5.8311980e-05]
18
Which is the most confident?
15
